# Объявление методов

Объявление метода похоже на объявление функции, но дополнительно появляется ещё один параметр перед названием функции. Этот параметр связывает метод со своим типом.

In [1]:
import "math"

type Point struct {
    X float64
    Y float64
}

// Обычная функция.
func Distance(p, q Point) float64 {
    return math.Hypot(q.X - p.X, q.Y - p.Y)
}

// Метод для типа Point.
func (p Point) Distance(q Point) float64 {
    return math.Hypot(q.X - p.X, q.Y - p.Y)
}

Дополнительный параметр p называют **получателем**(receiver) метода, наследство от первых ООП языков, в которых вызов метода назывался "отправкой сообщения объекту".

В Go не принято называть получатель специальным именем типа this или self. Он называется также как и обычный параметр функции. Т.к. имя получателя будет использоваться довольно часто, желательно подобрать ему короткое имя и использовать именно его во всех методах. Обычно выбирают **первую букву** типа как p для Point в нашем примере.

Для вызова метода сначала указывают получатель, а затем через точку метод:

In [2]:
import "fmt"

p1 := Point{1, 2}
p2 := Point{4, 6}
d := p1.Distance(p2) // Вызов метода.
fmt.Sprint(d)

5


Между объявлением функции Distance и методом Distance типа Point нет никакого конфликта: функция Distance является функцией уровня пакета, а метод принадлежит типу Point.

Выражение p.Distance называется **селектором**(selector), т.к. он выбирает метод Distance получателя p. Селекторы также используются для выбора полей, как например, p.X. Т.к. методы и поля находятся в одном и том же пространстве имён, нельзя давать им одинаковые имена.

Т.к. каждый тип определяет своё собственное пространство имён, можно объявлить метод Distance и для другого типа:

In [3]:
type Path []Point

func (p Path) Distance() float64 {
    sum := 0.0
    for i := range p {
        if i > 0 {
            sum += p[i-1].Distance(p[i])    
        }
    }
    return sum
}

Path является именованным типом, а не структурой как в случае с Point, но всё равно для него тоже можно определить метод. Иногда бывает удобно определить новые методы для простых типов как числовой, строковый и т.д., т.к. мы  получаем функционал этих базовых типов бесплатно. Методы можно объявить для любого именованного типа в этом же пакете, кроме типов, для которых базовым типом является указатель или интерфейс.